In [1]:
import numpy as np 
import pandas as pd 
import pandas as pd
from geopy.geocoders import Nominatim
import time
import requests

data = pd.read_csv("df_UK_lat_lon.csv")

In [ ]:
base_url = "http://environment.data.gov.uk/flood-monitoring/id/stations"
limit = 500  
offset = 0
station_ids = set()  
while True:
    url = f"{base_url}?_limit={limit}&_offset={offset}"
    response = requests.get(url)
    data = response.json()
    items = data.get("items", [])
    if not items: 
        break
    for item in items:
        station_id = item.get("stationReference")
        if station_id:
            station_ids.add(station_id)
    offset += limit
station_ids = list(station_ids)
data["stations_ids"] = station_ids

In [ ]:
station_ids

['024024',
 'E21747',
 '760502',
 '3766',
 'E1541',
 'E1366',
 'E13980',
 'SU47_141',
 '5518TH',
 'F1207',
 '3015TH',
 '3870TH',
 'L0237',
 '43228',
 '3687',
 'E23015',
 'E21604',
 '720300',
 '559969',
 '077336',
 '724830',
 '2056',
 '046222',
 '7059',
 '022014',
 '693132',
 'L32011',
 'E2578',
 'F1402',
 '45208',
 'E6630',
 'E14893',
 '713120',
 'L2024',
 'L3001',
 '50188',
 'E160',
 '050301012',
 'E21765',
 '2875A',
 'E5660',
 '53148',
 '4391TH',
 '067006',
 '3185',
 'E12570',
 '562417',
 '2667',
 '2170a',
 '567866',
 'E2728',
 'E6990',
 'E12412',
 '681213',
 'E4510',
 '2206A',
 'F1403',
 '45210',
 '010312',
 'L3503',
 '083534',
 '730100',
 '1771',
 '2153',
 'E22779',
 '561299',
 'E7451',
 'SJ45_155',
 'E82080',
 'SU48_71',
 'E12022',
 '2876B',
 'L2462',
 'E65601',
 '43177',
 'L1376',
 '2859TH',
 '4198',
 'L2907',
 '0184TH',
 'E24317',
 'F3123',
 '26103',
 '024010',
 '490001TP',
 '270669TP',
 '2420_TH',
 'E7071',
 'E6340',
 '3305SE',
 '2195',
 '3209TH',
 '5000',
 'E4832',
 '2513',
 '

In [ ]:
import requests 
import json
import numpy as np
import pandas as pd

data_request = requests.get('https://environment.data.gov.uk/flood-monitoring/id/stations')
read = data_request.json()
temp_lis = []
for i in read['items']:
    temp_lis.append(i)
columns = []
for j in i.keys():
    j = str(j)
    columns.append(j)

In [1]:
import json

with open('uk.geojson', 'r') as f:
    geojson_data = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'uk.geojson'

In [ ]:
data = temp_lis

flattened_data = []
for station in data:
    station_dict = {
        'id': station.get('@id'),
        'RLOIid': station.get('RLOIid'),
        'catchmentName': station.get('catchmentName'),
        'dateOpened': station.get('dateOpened'),
        'datumOffset': station.get('datumOffset'),
        'easting': station.get('easting'),
        'label': station.get('label'),
        'lat': station.get('lat'),
        'long': station.get('long'),
        'northing': station.get('northing'),
        'notation': station.get('notation'),
        'riverName': station.get('riverName'),
        'stageScale': station.get('stageScale'),
        'stationReference': station.get('stationReference'),
        'status': station.get('status'),
        'town': station.get('town'),
        'wiskiID': station.get('wiskiID'),
        'measures': json.dumps(station.get('measures', []))  # Convert measures list to string
    }
    flattened_data.append(station_dict)

df = pd.DataFrame(flattened_data)

print(df.head())

                                                  id RLOIid  \
0  http://environment.data.gov.uk/flood-monitorin...   7041   
1  http://environment.data.gov.uk/flood-monitorin...   6022   
2  http://environment.data.gov.uk/flood-monitorin...   3072   
3  http://environment.data.gov.uk/flood-monitorin...   6177   
4  http://environment.data.gov.uk/flood-monitorin...   2032   

                             catchmentName  dateOpened  datumOffset easting  \
0                                Cotswolds  1994-01-01          NaN  417990   
1                                  Welland  1992-01-01          2.0  528000   
2  Parrett, Brue and West Somerset Streams  1997-01-01         10.0  344383   
3                   Upper and Bedford Ouse  1996-10-01          6.3  529500   
4             Worcestershire Middle Severn  1976-04-06          NaN  386120   

             label        lat      long northing notation         riverName  \
0  Bourton Dickler  51.874767 -1.740083   219610   1029TH      Rive

In [ ]:
df.nunique()

TypeError: unhashable type: 'list'

In [ ]:
df.to_csv('df_UK_lat_lon.csv', index=False)

In [ ]:
data = pd.read_csv("df_UK_lat_lon.csv")

In [ ]:
data.columns

Index(['id', 'RLOIid', 'catchmentName', 'dateOpened', 'datumOffset', 'easting',
       'label', 'lat', 'long', 'northing', 'notation', 'riverName',
       'stageScale', 'stationReference', 'status', 'town', 'wiskiID',
       'measures'],
      dtype='object')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5392 entries, 0 to 5391
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                5392 non-null   object 
 1   RLOIid            2526 non-null   object 
 2   catchmentName     2526 non-null   object 
 3   dateOpened        2526 non-null   object 
 4   datumOffset       294 non-null    float64
 5   easting           4758 non-null   object 
 6   label             5392 non-null   object 
 7   lat               4758 non-null   object 
 8   long              4758 non-null   object 
 9   northing          4758 non-null   object 
 10  notation          5391 non-null   object 
 11  riverName         2526 non-null   object 
 12  stageScale        2526 non-null   object 
 13  stationReference  5391 non-null   object 
 14  status            2526 non-null   object 
 15  town              2367 non-null   object 
 16  wiskiID           2511 non-null   object 
